In [1]:
import sys
from pathlib import Path

# 현재 폴더: ptta/other_method/ActMAD/
# ptta 바로 위의 디렉토리를 sys.path에 추가
PROJECT_PARENT = Path.cwd().parents[0]  # -> ptta/ 의 부모 디렉토리
print(PROJECT_PARENT)
sys.path.insert(0, str(PROJECT_PARENT))

from os import path

import torch
from torch import nn, optim
from torch.utils.data import DataLoader

from ttadapters.datasets import BaseDataset, DatasetHolder, DataLoaderHolder
from ttadapters.datasets import SHIFTClearDatasetForObjectDetection, SHIFTCorruptedDatasetForObjectDetection, SHIFTDiscreteSubsetForObjectDetection

from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from accelerate import Accelerator, notebook_launcher

from supervision.metrics.mean_average_precision import MeanAveragePrecision
from supervision.detection.core import Detections

# import wandb
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

/workspace/ptta


## Check GPU Availability

In [2]:
!nvidia-smi

Sun Aug 31 11:11:30 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:04:00.0 Off |                    0 |
| N/A   39C    P0              34W / 250W |   1924MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
# Set CUDA Device Number 0~7
DEVICE_NUM = 6
ADDITIONAL_GPU = 0
DATA_TYPE = torch.bfloat16

if torch.cuda.is_available():
    if ADDITIONAL_GPU:
        torch.cuda.set_device(DEVICE_NUM)
        device = torch.device("cuda")
    else:
        device = torch.device(f"cuda:{DEVICE_NUM}")
else:
    device = torch.device("cpu")
    DEVICE_NUM = -1

print(f"INFO: Using device - {device}" + (f":{DEVICE_NUM}" if ADDITIONAL_GPU else ""))

INFO: Using device - cuda:6


In [4]:
PROJECT_NAME = "ActMAD test"
RUN_NAME = "RT-DETR_R50_ActMAD"

## Baseline

In [5]:
import baseline
data_root = path.normpath(path.join(Path.cwd(), "..", "data"))
print(data_root)
direct_method = baseline.Baseline(
    device=device,
    data_root=data_root,
    save_dir="./direct_method_checkpoints",
    method="direct_method",
    model_arch="rt_detr",
    reference_model_id="PekingU/rtdetr_r50vd",
    image_size=800,
    class_num=6,
    valid_batch=16,
    clean_bn_extract_batch=32,
    model_states="/workspace/ptta/RT-DETR_R50vd_SHIFT_CLEAR.safetensors",
    momentum=0.05,
    mom_pre=0.07,
    decay_factor=0.97,
    min_momentum_constant=0.01,
    patience=10,
    eval_every=10,
    enable_log_file=True
)

actmad = baseline.Baseline(
    device=device,
    data_root=data_root,
    save_dir="./actmad_checkpoints",
    method="actmad",
    model_arch="rt_detr",
    reference_model_id="PekingU/rtdetr_r50vd",
    image_size=800,
    class_num=6,
    valid_batch=16,
    clean_bn_extract_batch=16,
    model_states="/workspace/ptta/RT-DETR_R50vd_SHIFT_CLEAR.safetensors",
    momentum=0.05,
    mom_pre=0.07,
    decay_factor=0.97,
    min_momentum_constant=0.01,
    patience=10,
    eval_every=10,
    enable_log_file=True
)

norm = baseline.Baseline(
    device=device,
    data_root=data_root,
    save_dir="./norm_checkpoints",
    method="norm",
    model_arch="rt_detr",
    reference_model_id="PekingU/rtdetr_r50vd",
    image_size=800,
    class_num=6,
    valid_batch=16,
    clean_bn_extract_batch=16,
    model_states="/workspace/ptta/RT-DETR_R50vd_SHIFT_CLEAR.safetensors",
    momentum=0.05,
    mom_pre=0.07,
    decay_factor=0.97,
    min_momentum_constant=0.01,
    patience=10,
    eval_every=10,
    enable_log_file=True
)

dua = baseline.Baseline(
    device=device,
    data_root=data_root,
    save_dir="./dua_checkpoints",
    method="dua",
    model_arch="rt_detr",
    reference_model_id="PekingU/rtdetr_r50vd",
    image_size=800,
    class_num=6,
    valid_batch=16,
    clean_bn_extract_batch=16,
    model_states="/workspace/ptta/RT-DETR_R50vd_SHIFT_CLEAR.safetensors",
    momentum=0.05,
    mom_pre=0.07,
    decay_factor=0.97,
    min_momentum_constant=0.01,
    patience=10,
    eval_every=10,
    enable_log_file=True
)

/workspace/ptta/data


In [ ]:
direct_method.train()

In [ ]:
actmad.train()

In [ ]:
norm.train()

[2025-08-31 11:11:35,678] INFO: Log file: ./norm_checkpoints/norm_20250831_111135.log
[2025-08-31 11:11:35,791] INFO: start cloudy
[2025-08-31 11:11:35,928] INFO: INFO: Downloading 'SHIFT_SUBSET' from file server to /workspace/ptta/data/SHIFT/discrete...
[2025-08-31 11:11:35,929] INFO: INFO: Dataset archive found in the root directory. Skipping download.
[2025-08-31 11:11:35,930] INFO: INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
[2025-08-31 11:11:35,932] INFO: INFO: Downloading 'SHIFT_SUBSET' from file server to /workspace/ptta/data/SHIFT/discrete...
[2025-08-31 11:11:35,933] INFO: INFO: Dataset archive found in the root directory. Skipping download.
[08/31/2025 11:11:35] SHIFT DevKit - INFO - Base: /workspace/ptta/data/SHIFT/discrete/images/train. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7f25b442f250>
[08/31/2025 11:11:35] SHIFT DevKit - INFO - Loading annotation from '/workspace/ptta/data/SHIFT_SUBSET/cloudy_daytime/discrete/images/train

  0%|          | 0/13850 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

[2025-08-31 11:16:14,299] INFO: mAP@0.95_cloudy: 0.490
[2025-08-31 11:16:14,302] INFO: mAP50_cloudy: 0.663
[2025-08-31 11:16:14,304] INFO: mAP75_cloudy: 0.537
[2025-08-31 11:16:14,305] INFO: pedestrian_mAP@0.95_cloudy: 0.463
[2025-08-31 11:16:14,306] INFO: car_mAP@0.95_cloudy: 0.562
[2025-08-31 11:16:14,307] INFO: truck_mAP@0.95_cloudy: 0.558
[2025-08-31 11:16:14,308] INFO: bus_mAP@0.95_cloudy: 0.464
[2025-08-31 11:16:14,309] INFO: motorcycle_mAP@0.95_cloudy: 0.446
[2025-08-31 11:16:14,310] INFO: bicycle_mAP@0.95_cloudy: 0.446
[2025-08-31 11:16:14,385] INFO: [cloudy] batch 0: mAP50 -1.0000 -> 0.6630 ✔


  0%|          | 0/150 [00:00<?, ?it/s]

In [ ]:
dua.train()